# Daten einlesen

In [1]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\tlock\miniconda3\envs\Opt1\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [96]:
dat_mat = scipy.io.loadmat("TrafficDataset\sequence_data1-1_1.mat")
dat_mat.keys()

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\tlock\AppData\Local\Temp\ipykernel_16468\1656983495.py:1: SyntaxWarning: invalid escape sequence '\s'
  dat_mat = scipy.io.loadmat("TrafficDataset\sequence_data1-1_1.mat")


dict_keys(['__header__', '__version__', '__globals__', 'sequences'])

## repackage data

In [97]:
data_array = np.array(dat_mat["sequences"])
data = []
for part in data_array:
    data.append(part[0][0][0][0])

ich verstehe nicht wirklich was die daten sein sollen?

In [98]:
# Kosten funktional
def J (x, dt, x_data):
    result = 0
    for i in range(len(x)):
        dt * np.abs(x[i] - x_data[i])**2
    return result

In [29]:
# Den mittlwert von J jederzeit ausgeben können:
class J_mittel():
    __val = 0
    __n = 0
    def set_val(self, x):
        self.__val = x
        return
    def set_n(self, x):
        self.__n = x
        return
    
    def get_val(self):
        return self.__val
    def get_n(self):
        return self.__n
    
    def get_res(self):
        return self.__val/self.__n

    def __init__(self, x):
        self.__val = x
        self.__n = 1
        return
    
    def add(self, x):
        self.__val += x
        self.__n += 1
        return

J = J_mittel(0)
for i in range(1, 11):
    J.add(i)

J.get_res()

5.0

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import pygame

# Kosten funktional
def J (x, dt, x_data):
    result = 0
    for i in range(len(x)):
        dt * np.abs(x[i] - x_data[i])**2
    return result

def f (d, v_max, model = "log"):
    if model == "log":
        return v_max * np.log(d)
    else: # lin
        return v_max * (1 - 1/d)
    
# Den mittlwert von J jederzeit ausgeben können:
class J_mittel():
    __val = 0
    __n = 0
    def set_val(self, x):
        self.__val = x
        return
    def set_n(self, x):
        self.__n = x
        return
    
    def get_val(self):
        return self.__val
    def get_n(self):
        return self.__n
    
    def get_res(self):
        return self.__val/self.__n

    def __init__(self, x):
        self.__val = x
        self.__n = 1
        return
    
    def add(self, x):
        self.__val += x
        self.__n += 1
        return

def d(x, max_W = 2000, loop = True): # Abstandsfunktion, damit immer des nächste auto VOR dem derzeitig betrachteten auto berücksichtigt wird
    d = np.ones_like(x) * 10**10 # anfangs distanz auf sehr groß setzen
    for i in range(len(x)):
        for j in range(len(x)):
            if i == j:
                continue
            else:
                d[i] = x[j]-x[i] if d[i] > x[j]-x[i] and x[j]-x[i] > 0 else d[i]
            if loop and i == len(x)-1: # abnhängig von der breite der anzeige wird nun der abstand zu den anderen autos überprüft
                d[i] = x[j]-((x[i]-max_W)) if d[i] > x[j]-(x[i]-max_W) and x[j]-(x[i]-max_W) > 0 else d[i]
    return d

def single_step(x, v_max, L, dt, model, max_W):
    N = len(x) # Number of initial values == number of vehicles in traffic
    dx = np.zeros(N) # momente initialisieren
    # Leader
    # dx[-1] = v_max
    dist = d(x, max_W)
    for i in range(N):
            
            dx[i] = f(dist[i], v_max, model)
        
    return x + dt * dx, dx

def simulation_screen(x0, v_max, L, dt, model):
    pygame.init()
    W, H = 1500, 200
    screen = pygame.display.set_mode((W, H))
    clock = pygame.time.Clock()
    scale = 10     # skaliert Meter → Pixel
    running = True
    x = np.array(x0, dtype=float)
    while running:
        clock.tick(60) # 60 FPS
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False             # Stop when closed
        
        x, v = single_step(x, v_max, L, dt, model, max_W = W)

        screen.fill((30, 30, 30))

        for i in range(len(x)):
            px = int(x[i]*scale)% W # start again from the beginning of the screen if done with the lap
            py = H // 2 # bewegt sich auf der hälfter der screen höhe
            if i == len(x)-1:
                pygame.draw.circle(screen, (255, 200, 100), (px, py), 8) #cars
            else:
                pygame.draw.circle(screen, (100, 200, 255), (px, py), 8) #cars

            #vx = int(v[i] *0.4) # 0.4 skarilerungs faktor für weniger clutter
            #pygame.draw.line(screen, (255, 160, 60), (py, py), (px+vx, py), 3)  #geschwindigkeits indikator linie
            # Geschwindigkeitspfeil
            vx = int(v[i] * 2)     # skaliert für Anzeige
            pygame.draw.line(screen, (255, 160, 60), (px, py), (px + vx, py), 3)

        pygame.display.flip() # display and render 
    pygame.quit()

In [ ]:
# Parameter
N = 4 # Anzahl autos
FPS = 60
scale_m_zu_px = 10
v_kmh = 100
v_max = v_kmh * scale_m_zu_px / FPS * 1000/3600 # km/h -> m/s
Sicherheitsabstand_m = 50 
L =  Sicherheitsabstand_m * scale_m_zu_px
x0 = np.arange(100, 100+40*N, step=40)      # Startpositionen

print((v_max, L, x0))
simulation_screen(x0, v_max, L, dt=1/FPS, model="log")
